In [3]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', None)

In [30]:
content=pd.read_csv('data/Content.csv',chunksize=5000)
compound=pd.read_csv('data/Compound.csv')
compound=compound[['id','name','cas_number']]
drugdb=pd.read_csv('data/Drug_info_combined.csv')

/Users/zhaoxiangchen/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2728: DtypeWarning: Columns (3,4,5,6,12,13,14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [31]:
food_compound=pd.DataFrame()
for i in content:
    i=i.loc[i['source_type']=='Compound']
    i=i.loc[i['orig_content']>0]
    if len(i)>0:
        i=i.merge(compound,left_on='source_id',right_on='id',how='left')
        i=i.dropna(subset=['cas_number','orig_food_common_name'])
        i['orig_food_common_name']=i['orig_food_common_name'].apply(lambda x: x.split(',')[0])
        i=i.groupby('orig_food_common_name')['orig_food_common_name','orig_source_name','orig_content','cas_number'].apply(lambda x: x.nlargest(3,'orig_content'))
        i=i.reset_index(drop=True)
        food_compound=pd.concat([food_compound,i])
food_compound=food_compound.dropna()
food_compound=food_compound.groupby('orig_food_common_name')['orig_food_common_name','orig_source_name','orig_content','cas_number'].apply(lambda x: x.drop_duplicates('orig_source_name'))
food_compound.reset_index(drop=True,inplace=True)
food_compound=food_compound.groupby('orig_food_common_name')['orig_food_common_name','orig_source_name','orig_content','cas_number'].apply(lambda x: x.nlargest(3,'orig_content'))
food_compound.reset_index(drop=True,inplace=True)

In [35]:
food_compound=pd.read_csv('data/Food_compounds.csv')

In [57]:
food_compound[food_compound['orig_food_common_name']=='Apple']['orig_source_name'].values

array(['Sugars, total', 'SUCROSE|SUGAR', 'FRUCTOSE'], dtype=object)